In [ ]:
from langchain_openai import OpenAI
from langchain.agents import load_tools
from langchain.agents import AgentExecutor
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.tools.render import render_text_description
from langchain import hub

# 通过python-dotenv加载环境变量
from dotenv import load_dotenv
load_dotenv()

# 准备大语言模型：这里需要使用OpenAI()，可以方便地按需停止推理
llm = OpenAI()
llm_with_stop = llm.bind(stop=["\nObservation"])

# 准备工具：这里用到DuckDuckGo搜索引擎和一个基于LLM的计算器
tools = load_tools(['ddg_search', 'llm-math'], llm=llm)

# 准备核心提示词：这里从LangChain Hub加载了ReAct模式的提示词，并且填充工具的文本描述
prompt = hub.pull('hwchase17/react')
prompt = prompt.partial(
    tools = render_text_description(tools),
    tool_names = ', '.join([t.name for t in tools]),
)

# 构建Agent工作链：这里最重要的是，把中间步骤的结构保存到提示词的agent_scratchpad中
agent = (
    {
        'input': lambda x: x['input'],
        'agent_scratchpad': lambda x: format_log_to_str(x['intermediate_steps']),
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser
)

# 构建Agent执行器：执行器负责执行Agent工作链，直至得到最终答案（的标识）并输出回答
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({'input': '今天哈尔滨和汉中的气温相差几摄氏度？'})